# AVI 2

This is for first steps... of AVI by using Jupyter

https://avinetworks.com/docs/latest/api-guide/overview.html 


https://github.com/avinetworks/sdk/blob/master/python/avi/sdk/README.md

[1.- Initial Login](#1--initial-login)
[2.- Read Operations](#import-libraries)

## Initial Login

In [ ]:
from avi.sdk.avi_api import ApiSession
import datetime, time
from requests.packages import urllib3
urllib3.disable_warnings()
import json

# Import environment variables with controller information and credentials
from envs.controller_info import session_params as session_env

# Establish a first session with AVI Controller
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Update headers and api version imported from demo env file with controller version (ensure actual API Version is uses in subsequent requests
session_env['headers']['X-Avi-Version'] = api.remote_api_version['Version']
session_env['api_version'] = api.remote_api_version['Version']

# Create a new session with received AVI API Version
api = ApiSession(
    controller_ip=session_env['controller_ip'],
    username=session_env['controller_username'],
    password=session_env['controller_password'],
    tenant=session_env['tenant'],
    api_version=session_env['api_version']
    )
# Display Session ID to Verify AVI Controller Session Establishment
print('Successful connection to ' + session_env['name'] + '. Session ID:' + api.session_id)

## 2.- Read Operations

Get is a common method to read information.
You can get information of your current setup easily invoking the corresponding API

AVI has dozens of APIs... 
   - Use https://controller/swagger to understand different API
   - You can easily infere the API when navigating through the GUI just looking at the address bar used in the GUI 

   Example if the GUI shows https://192.168.1.15/#!/admin/applications/virtualservice that means you are in the **admin** tenant and you are using the **virtualservice** API

   In the previous step we created an AVI API session object that we called *api*. 
   Let's invoke the get method over our api object to inspect the output

### 2.1 Basic Operations via JSON

Import some required Libraries first

In [ ]:
# Use Pandas to work with dataframes. If not installed use pip install pandas first
import pandas as pd

Now create a new object named vs with will contain the output of the get method using the virtualservice API path. 
We will use here the skip_default query parameter set to true to save only relevant information.

In [ ]:
query = {
   "skip_default": "true"
}
vs = api.get('virtualservice', params=query)
data = json.loads(vs.text)

print ("Sending request to following URL: " +vs.url)
data

The request produces a json formatted output. You can change it by using 

Saving the response in a document using json format (double quotes)

In [ ]:
# Save into a file
with open('output.json', 'w') as file:
    file.write(repr(data))

In [ ]:
#Open JSON file (double Quotes) and load contents into Python Dictionary (single quotes)
file_path = 'sample_outputs/serviceenginegroup_output.json'
with open(file_path, 'r') as file:
    serviceenginegroup_data = json.load(file)

serviceenginegroup_data

In [ ]:
se_data = serviceenginegroup_data["results"][0]

In [ ]:
se_data

Filtering to include only interesting/mandatory fields

In [ ]:
fields_of_interest = [
    'algo', 
    'vs_se_scaleout_ready_timeout', 
    'active_standby']
filtered_data = {key: se_data[key] for key in fields_of_interest if key in se_data }
filtered_data


In [ ]:
# import results into a dataframe
df=pd.json_normalize(data['results'])
# Display the result of the df dataframe object
df

In [ ]:
# Save result into a csv file  
df.to_csv('output_vs_inventory.csv', encoding='utf-8')

## AVI Post 

In [ ]:
# Cloning an object from an exising one after changing required parameters

# Define GET parameters
url_path="serviceenginegroup"
src_name = "SOURCE-SEG001"
query = {
   "skip_default": "false",
   "name": src_name 
}

resp = api.get(url_path, params=query)
print ("Sending request to following URL: " +resp.url)

if resp.status_code in range(200, 299):
    print(resp)
    print(resp.reason)
else:
    print('Error in GET request '+url_path+' :%s' % resp.text)

data = json.loads(resp.text)

# Extracting results contents
data = data["results"][0]
data

https://avinetworks.com/docs/latest/api-guide/ 


In [ ]:
# Filtering to extract relevant fields
fields_of_interest = [
    'name', 
    'cloud_ref', 
    'max_vs_per_se']
data = {key: data[key] for key in fields_of_interest if key in data }
data

In [ ]:
# Define new parameters
new_fields = {
   "name": "SEG-GROUP-003",
   "max_vs_per_se": 8
}
data.update(new_fields)

In [ ]:
data

In [ ]:
#Send POST information via POST

url_path = "serviceenginegroup"

resp = api.post (url_path, data=json.dumps(data))

if resp.status_code in range(200, 299):
    print(resp)
    print('- New '+url_path+' named '+data['name'], resp.reason)#, resp.text)
else:
    print('Error in creating '+url_path+' :%s' % resp.text)